### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
print(groq_api_key)

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


In [2]:
from langchain_groq import ChatGroq # using groq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026489C1A0C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026489C1B260>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [4]:
result

AIMessage(content='Bonjour, comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 21, 'total_tokens': 32, 'completion_time': 0.02, 'prompt_time': 0.002182096, 'queue_time': 0.229376249, 'total_time': 0.022182096}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5a20fa32-4027-4a35-a66a-5fe2b6460b40-0', usage_metadata={'input_tokens': 21, 'output_tokens': 11, 'total_tokens': 32})

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

In [6]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'**Bonjour**\n\n**Comment allez-vous ?** \n\n\nYou can also use "Comment vas-tu?" if you are speaking to someone you know well.\n'

In [7]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [8]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [9]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [10]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

"Bonjour \n\n\nLet me know if you'd like to translate anything else!\n"